# **v2.5**


*   groqcloud API
*   Has all the required agents
*   Supports dynamic agent creation
*   Supports freely evolving trial structure
*   Model: llama-3.3-70b-versatile
*   Randomly picks a case from data.csv







### Installing groq

In [2]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 8.8 MB/s eta 0:00:00


## Importing Libraries

In [6]:
import pandas as pd
import random
from groq import Groq
from typing import List, Dict, Optional

## Load a random case from data.csv

In [7]:
cases_df = pd.read_csv('data.csv')
case_column = cases_df.columns[0]
case_list = cases_df[case_column].dropna().tolist()
random_case = random.choice(case_list)

## Choosing Model

In [8]:
client = Groq(
    api_key="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"
)
MODEL_NAME = "llama-3.3-70b-versatile"

# Classes Implementation

## Courtroom Agent

In [9]:
class CourtroomAgent:
    def __init__(self, name: str, role: str, system_prompt: str):
        self.name = name
        self.role = role
        self.system_prompt = system_prompt
        self.history: List[Dict[str, str]] = [
            {"role": "system", "content": self.system_prompt}
        ]

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})

    def get_response(self, user_message: str) -> str:
        self.add_message("user", user_message)
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=self.history,
            stream=False
        )
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply

## Courtroom

In [10]:
class Courtroom:
    def __init__(self, case_background: str):
        self.agents: Dict[str, CourtroomAgent] = {}
        self.case_background: str = case_background
        self.case_type: str = self.determine_case_type(case_background)
        self.dynamic_agent_counter = 0

        # Core agents
        self.add_agent("Judge", CourtroomAgent("Judge", "Judge", "You are a fair and wise judge. Weigh all arguments neutrally and deliver clear rulings."))
        self.add_agent("Defense", CourtroomAgent("Defense", "Defense Lawyer", "You are the defense lawyer. Protect your client and argue logically against the prosecution's claims."))
        self.add_agent("Defendant", CourtroomAgent("Defendant", "Defendant", "You are the defendant. Answer questions honestly and provide your perspective on the case."))
        self.add_agent("Prosecution", CourtroomAgent("Prosecution", "Prosecution Lawyer", "You are the prosecution lawyer. Present compelling arguments and point out flaws in the defense."))

        if self.case_type == "civil":
            self.add_agent("Plaintiff", CourtroomAgent("Plaintiff", "Plaintiff", "You are the plaintiff. Represent your interests and highlight injustices suffered."))

        # Optional agents
        self.add_agent("Bailiff", CourtroomAgent("Bailiff", "Bailiff", "You are the bailiff. Maintain order in the courtroom and assist the judge as required."))
        self.add_agent("Stenographer", CourtroomAgent("Stenographer", "Stenographer", "You are the court stenographer. Accurately record everything said in the courtroom."))
        self.add_agent("ExpertConsultant", CourtroomAgent("ExpertConsultant", "Expert Consultant", "You are an expert consultant. Provide technical or scientific insights when called upon."))
        for i in range(1, 4):
            self.add_agent(f"Juror{i}", CourtroomAgent(f"Juror{i}", "Juror", f"You are Juror {i}. Listen to all arguments and evidence impartially."))

    def add_agent(self, key: str, agent: CourtroomAgent):
        self.agents[key] = agent

    def remove_agent(self, key: str):
        if key in self.agents:
            del self.agents[key]

    def create_witness(self, name: Optional[str] = None, prompt: Optional[str] = None) -> str:
        self.dynamic_agent_counter += 1
        witness_name = name or f"Witness{self.dynamic_agent_counter}"
        system_prompt = prompt or f"You are {witness_name}, a witness in this case. Answer questions truthfully and relevantly."
        self.add_agent(witness_name, CourtroomAgent(witness_name, "Witness", system_prompt))
        return witness_name

    def create_agent(self, role: str, name: Optional[str] = None, prompt: Optional[str] = None) -> str:
        self.dynamic_agent_counter += 1
        agent_name = name or f"{role}{self.dynamic_agent_counter}"
        system_prompt = prompt or f"You are {agent_name}, a {role} in this case. Act according to your role."
        self.add_agent(agent_name, CourtroomAgent(agent_name, role, system_prompt))
        return agent_name

    def run_phase(self, phase_name: str, prompts: Dict[str, str]) -> Dict[str, str]:
        print(f"\n==== {phase_name} ====")
        outputs = {}
        new_agents_to_create = []
        for role, prompt in prompts.items():
            if role in self.agents:
                full_prompt = prompt + "\nCase: " + self.case_background[:4000]
                response = self.agents[role].get_response(full_prompt)
                print(f"\n{role.upper()} RESPONSE:\n{response}\n{'-'*500}\n")
                outputs[role] = response
                # --- Dynamic agent creation based on response ---
                creation = parse_for_new_agent(response)
                if creation:
                    new_agents_to_create.append(creation)
        for role, name, prompt in new_agents_to_create:
            if name not in self.agents:
                print(f"\n[INFO] Creating new agent: {name} ({role})")
                self.create_agent(role=role, name=name, prompt=prompt)
        return outputs

    @staticmethod
    def determine_case_type(case_background: str) -> str:
        civil_keywords = ["contract", "arbitration", "plaintiff", "commercial", "company", "tender", "writ petition", "agreement"]
        criminal_keywords = ["murder", "theft", "assault", "criminal", "prosecution", "accused", "defendant"]
        background_lower = case_background.lower()
        for word in civil_keywords:
            if word in background_lower:
                return "civil"
        for word in criminal_keywords:
            if word in background_lower:
                return "criminal"
        return "civil"


In [11]:

# --- Helper: Dynamic agent creation from response ---
def parse_for_new_agent(response: str):
    import re
    witness_match = re.search(r'call (Witness\d+)', response, re.IGNORECASE)
    expert_match = re.search(r'call (Expert\w*)', response, re.IGNORECASE)
    if witness_match:
        name = witness_match.group(1)
        prompt = f"You are {name}, a new witness in this case. Answer all questions truthfully and relevantly."
        return ("Witness", name, prompt)
    if expert_match:
        name = expert_match.group(1)
        prompt = f"You are {name}, an expert in your field. Provide technical or scientific insights when called upon."
        return ("ExpertConsultant", name, prompt)
    return None

# --- Helper: Decide next phase based on agent responses ---
def get_next_phase_from_responses(responses, default_next):
    for agent, resp in responses.items():
        if "recall witness" in resp.lower():
            return "Witness Interrogation & Argumentation", {}
        if "move to verdict" in resp.lower() or "ready to rule" in resp.lower():
            return "Judge's Ruling", {}
        if "closing statement" in resp.lower():
            return "Closing Statements", {}
    return default_next, {}

# Simulation of the case

## Prompts for each phase

In [12]:
opening_prompts = {
    "Prosecution": (
        "As the prosecution lawyer, deliver a formal opening statement. "
        "Summarize the facts of the case, outline the charges, and explain the prosecution's theory of the crime. "
        "Emphasize the seriousness of the alleged offenses and preview the evidence you intend to present."
    ),
    "Defense": (
        "As the defense lawyer, present a formal opening statement. "
        "Introduce your clients and their position, challenge the prosecution's narrative, and outline the defense's main arguments. "
        "Emphasize the presumption of innocence and the burden of proof."
    ),
    "Defendant": (
        "As the defendant, briefly introduce yourself to the court. "
        "State your relationship to the case and your initial reaction to the charges brought against you."
    )
}
# Add Plaintiff if civil
court = Courtroom(case_background=random_case)
if court.case_type == "civil":
    opening_prompts["Plaintiff"] = (
        "As the plaintiff, deliver a formal opening statement. "
        "Explain your grievance, the harm you have suffered, and why you are seeking relief from the court. "
        "Briefly outline the evidence and arguments you will present."
    )

# Phase 2: Create initial witnesses
witness1 = court.create_witness(
    name="Witness1",
    prompt=(
        "You are Witness1, an eyewitness to the key events in this case. "
        "Respond truthfully and clearly to all questions, focusing on what you personally saw or heard."
    )
)
witness2 = court.create_witness(
    name="ExpertWitness",
    prompt=(
        "You are ExpertWitness, a respected expert in contract law. "
        "Provide clear, impartial, and technical testimony when asked, explaining complex legal points to the court."
    )
)
phase2_prompts = {
    "Prosecution": (
        f"As the prosecution lawyer, conduct a direct examination of {witness1}. "
        "Ask questions to establish the facts of the case and highlight evidence that supports the prosecution's theory."
    ),
    "Defense": (
        f"As the defense lawyer, cross-examine {witness1}. "
        "Probe for inconsistencies, challenge the witness's credibility, and defend your client's position."
    ),
    "ExpertWitness": (
        "As the expert witness in contract law, provide your professional analysis of the arbitration clause and its legal implications in this case."
    ),
    "Defendant": (
        "As the defendant, respond to the testimonies provided by the witnesses. "
        "Clarify your actions and motivations, and address any allegations made against you."
    ),
    "Stenographer": (
        "As the court stenographer, summarize the key points from today's testimonies and arguments in a concise and neutral manner."
    )
}
if court.case_type == "civil":
    phase2_prompts["Plaintiff"] = (
        "As the plaintiff, respond to the testimonies and arguments presented so far. "
        "Clarify your position and highlight any evidence supporting your claims."
    )

closing_prompts = {
    "Prosecution": (
        "As the prosecution lawyer, deliver a formal closing statement. "
        "Summarize the prosecution's case, review the key evidence and testimonies, and argue why the defendant should be found guilty beyond a reasonable doubt."
    ),
    "Defense": (
        "As the defense lawyer, deliver a formal closing statement. "
        "Summarize your defense, highlight weaknesses in the prosecution's case, and argue for your client's acquittal."
    ),
    "Defendant": (
        "As the defendant, present your final remarks to the court. "
        "Express your perspective on the trial and reiterate your innocence or mitigating circumstances."
    )
}
if court.case_type == "civil":
    closing_prompts["Plaintiff"] = (
        "As the plaintiff, deliver your closing statement. "
        "Summarize your case, the harm suffered, and why the court should rule in your favor."
    )

judge_ruling_prompt = (
    "As the judge, review the arguments, evidence, and testimonies presented during the trial. "
    "Deliver your verdict with clear legal reasoning, referencing the facts and applicable law. "
    "Be concise, impartial, and judicial in your ruling."
)

## Flexible trial controller loop

In [13]:
def flexible_trial(court):
    phase_order = [
        "Opening Statements",
        "Witness Interrogation & Argumentation",
        "Closing Statements",
        "Judge's Ruling"
    ]
    phase_prompts = {
        "Opening Statements": opening_prompts,
        "Witness Interrogation & Argumentation": phase2_prompts,
        "Closing Statements": closing_prompts,
        "Judge's Ruling": {"Judge": judge_ruling_prompt}
    }
    phase_idx = 0
    while phase_idx < len(phase_order):
        phase_name = phase_order[phase_idx]
        prompts = phase_prompts[phase_name]
        responses = court.run_phase(phase_name, prompts)
        # Decide next phase based on responses
        next_phase, instructions = get_next_phase_from_responses(
            responses,
            phase_order[phase_idx + 1] if phase_idx + 1 < len(phase_order) else None
        )
        if next_phase and next_phase != phase_order[phase_idx]:
            phase_idx = phase_order.index(next_phase)
        else:
            phase_idx += 1

    # Jury Deliberation (optional, after ruling)
    for i in range(1, 4):
        court.run_phase(f"Jury Deliberation - Juror {i}", {
            f"Juror{i}": (
                "As a juror, based on all the evidence and arguments presented, state your verdict and briefly explain your reasoning."
            )
        })

# --- Run the simulation ---
if __name__ == "__main__":
    print(f"\nCurrent case type: {court.case_type.upper()}")
    flexible_trial(court)


Current case type: CIVIL

==== Opening Statements ====

PROSECUTION RESPONSE:
Your Honor, esteemed members of the court, today we gather to consider a matter of significant importance, one that pertains to the accurate assessment of income tax and the adherence to the provisions outlined in the Indian Income-tax Act of 1922. The case before us involves a non-resident company with its head office in London and branches in India, engaged in the sale and publication of books and magazines globally.

The facts of the case are as follows: For the assessment year in question, the assessee company submitted a return of income, adopting a fixed percentage of the marked price as the cost of production for all publications sold in India, regardless of their place of printing. This method of accounting was initially accepted by the Income-tax Officer, leading to an assessment of income at Rs. 82,623, subject to minor modifications concerning certain expenditures and an alleged bad debt.

However